# From Part 2:

In [1]:
import copy
import math
import sys

In [2]:
def smoothEmission(k,lines,lines2):
    x = {}
    Lines = copy.deepcopy(lines)
    Lines2= copy.deepcopy(lines2)
    for i in range (len(lines)):
        if lines[i]!='\n':
            ls = lines[i].split(" ")
            if ((ls[0]) not in x):
                    x[ls[0]] = 1
            else:
                    x[ls[0]] +=1
    print("step1 done")
    
    
    for i in range (len(lines)):
        if lines[i]!='\n':
            ls = lines[i].split(" ")
            if(x[ls[0]]<k):
                Lines[i]='unk '+ls[1]
                
    print("step2 done")
    
    for i in range(len(lines2)):
        if lines2[i]!='\n':
            if lines2[i].strip("\n") not in x:
                Lines2[i] = 'unk'
    print("step3 done")
    return(Lines,Lines2)

In [3]:
def emission(lines):
    xy={}
    y={}
    emi={}
    bg = 0
    for i in range (len(lines)):
        #print(bg)
        if lines[i]!='\n':
            ls = lines[i].split(" ")
            y_ = ls[1].strip('\n')
            if ((ls[0],y_) not in xy):
                        xy[(ls[0],y_)] = 1
            else:
                        xy[(ls[0],y_)]+=1

            if (y_ not in y):
                        y[y_]= 1
            else:
                        y[y_]+=1
            bg+=1
    for j in xy.keys():
                    emi[j]=(xy[j]/y[j[1]])
                    
    return(emi)

# Part 3:

In [4]:
def transition(path):
    # q(u -> v) = count(u,v)/count(u)
    
    u_label = {}
    uv_label = {}
    
    prevState = ''
    currState = '***START***'
    
    for line in open(path, 'r'):
        
        if currState == '***STOP***':
            prevState = '***START***'
        else:
            prevState = currState
            
        line = line.rstrip()
        xy = line.split(' ')
        
        if len(xy) > 1:
            x, currState = xy
            
        else:
            if prevState == '##START##': break
            currState = '***STOP***'
            if currState in u_label:
                u_label[currState] += 1
            else:
                u_label[currState] = 1
            
        if prevState in u_label:
            u_label[prevState] += 1
        else:
            u_label[prevState] = 1

        if (prevState, currState) in uv_label:
            uv_label[(prevState,currState)] += 1

        else:
            uv_label[(prevState,currState)] = 1
            
        q = {}
                
        for i, count in u_label.items():
            for j in u_label:
                if (i,j) in uv_label:
                    countuv = uv_label[(i,j)]
                    q[(i,j)] = countuv/count
                    
                
    return list(u_label.keys()), q

In [5]:
def viterbi(states, emission, transition, sentence):
    e = emission
    t = transition
    n = len(sentence)
    
    smallest = math.log(sys.float_info.min)-1
    
    #initialize score dict
    scores = {}
    
    #i=0
    scores[0] = {}
    for j in states:
        if ("***START***", j) in t:
            trans = math.log(t[("***START***", j)])
        else:
            trans = smallest
        if (sentence[0], j) in e:
            emis = math.log(e[(sentence[0], j)])
        else:
            emis = smallest
        
        ans = trans + emis
        scores[0][j] = (ans, "***START***")
    
    
    for i in range(1, n):
        scores[i] = {}
        for j in states:
            findmax = []
            for l in states:
                if (l,j) in t:
                    trans = math.log(t[(l,j)])
                else:
                    trans = smallest
                if (sentence[i],j) in e:
                    emis = math.log(e[(sentence[i],j)])
                else:
                    emis = smallest
                score = scores[i-1][l][0] + trans + emis
                findmax.append(score)

            ans = max(findmax)
            state_ans = states[findmax.index(ans)]
            scores[i][j] = (ans, state_ans)
        
            
    #STOP state
    scores[n] = {}
    findmax = []
    for j in states:
        if (j,"***STOP***") in t:
            trans = math.log(t[(j,"***STOP***")])
        else:
            trans = smallest
        score = scores[n-1][j][0] + trans
        findmax.append(score)
        
    stop = max(findmax)
    state_ans = states[findmax.index(stop)]
    scores[n] = (stop, state_ans)
    
    #backtracking
    path = ['***STOP***']
    last = scores[n][1]
    path.append(last)
    
    for k in range((n-1), -1, -1):
        last = scores[k][last][1]
        path.append(last)
                

    return scores, list(reversed(path))

# Part 4

In [6]:
def Nviterbi(N, states, emission, transition, sentence):
    e = emission
    t = transition
    n = len(sentence)
    
    smallest = math.log(sys.float_info.min)-1
    
    #initialize score dict
    scores = {}
    
    #i=0
    scores[0] = {}
    for j in states:
        if ("***START***", j) in t:
            trans = math.log(t[("***START***", j)])
        else:
            trans = smallest
        if (sentence[0], j) in e:
            emis = math.log(e[(sentence[0], j)])
        else:
            emis = smallest
        
        ans = trans + emis
        scores[0][j] = (ans, "***START***")
    
    
    for i in range(1, n):
        scores[i] = {}
        for j in states:
            findmax = []
            for l in states:
                if (l,j) in t:
                    trans = math.log(t[(l,j)])
                else:
                    trans = smallest
                if (sentence[i],j) in e:
                    emis = math.log(e[(sentence[i],j)])
                else:
                    emis = smallest
                if i > 1:
                    for m in range(N):
                        score = scores[i-1][l][m][0] + trans + emis
                        findmax.append((score,l,m))
                else:
                    score = scores[i-1][l][0] + trans +emis
                    findmax.append((score,l,0))

            #get top N scores
            topN = []
            for q in range(N):
                max1 = (-sys.maxsize, 'nan', 'nan')
                for w in range(len(findmax)):
                    if findmax[w][0] > max1[0]:
                        max1 = findmax[w]
                findmax.remove(max1)
                topN.append(max1)
            
            scores[i][j] = topN
        
            
    #STOP state
    scores[n] = {}
    findmax = []
    for j in states:
        if (j,"***STOP***") in t:
            trans = math.log(t[(j,"***STOP***")])
        else:
            trans = smallest
        for m in range(N):
            score = scores[n-1][j][m][0] + trans
            findmax.append((score, j, m))

    #get top N scores
    topN = []
    for q in range(N):
        max1 = (-sys.maxsize, 'nan', 'nan')
        for w in range(len(findmax)):
            if findmax[w][0] > max1[0]:
                max1 = findmax[w]
        findmax.remove(max1)
        topN.append(max1)
        
    scores[n] = topN
    
    
#     #backtracking
    path = ['***STOP***']
    last = scores[n][N-1]
    path.append(last[1])
    
    for k in range((n-1), 0, -1):
        last = scores[k][last[1]][last[2]]
        path.append(last[1])
        
    path.append('***START***')

    return scores, list(reversed(path))

# to run the functions:

In [19]:
#get lines of training data
r1 =  open("/Users/khai/Home Documents/Term 6/Machine Learning/AL/train","r",encoding='utf-8')
lines1 = r1.readlines()
r1.close()

In [20]:
#get lines of test data
r2 =  open("/Users/khai/Home Documents/Term 6/Machine Learning/AL/dev.in","r",encoding='utf-8')
lines2 = r2.readlines()
r2.close()

In [21]:
#smooth the data
train, test = smoothEmission(3, lines1, lines2)

step1 done
step2 done
step3 done


In [22]:
#clean data
x = train
for i in range(len(x)):
    if x[i] != '\n':
        x[i]= x[i].rstrip('\n')
        
y = test
biglist = []
temp = []
for i in range(len(y)):
    if y[i] != '\n':
        y[i]= y[i].rstrip('\n')
        
for j in y:
    if j != "\n":
        temp.append(j)
    else:
        biglist.append(temp)
        temp = []

In [23]:
#get emission, transitions and states
e = emission(train)
labels, q = transition("/Users/khai/Home Documents/Term 6/Machine Learning/AL/train")

In [24]:
lab = copy.deepcopy(labels)
lab.remove('***START***')
lab.remove('***STOP***')

# viterbi with whole training and test set

In [25]:
biglist

[['杭',
  '州',
  '市',
  '西',
  '湖',
  '区',
  '古',
  '荡',
  '新',
  '村',
  '西',
  '34',
  '幢',
  '八',
  '单',
  '元',
  '1375'],
 ['塘',
  '雅',
  '镇',
  '顶',
  '塘',
  '工',
  '业',
  '园',
  '鑫',
  '龙',
  '水',
  '准',
  '仪',
  '器',
  '有',
  '限',
  '公',
  '司'],
 ['中',
  '国',
  '浙',
  '江',
  '省',
  '温',
  '州',
  '市',
  '鹿',
  '城',
  '区',
  '百',
  '里',
  '东',
  '路',
  '47',
  '号'],
 ['浙',
  '江',
  '省',
  '宁',
  '波',
  '市',
  '宁',
  '海',
  '县',
  '宁',
  '海',
  '县',
  '_',
  '长',
  '街',
  '镇',
  '_',
  '手',
  '机',
  '号'],
 ['浙',
  '江',
  '省',
  '杭',
  '州',
  '余',
  '杭',
  '区',
  '乔',
  '司',
  '街',
  '道',
  '和',
  '睦',
  '村',
  '8',
  '组',
  '149',
  '号'],
 ['浙', '江', '省', '临', '安', '市', '太', '阳', '镇', '上', '庄', '村', '博', '远'],
 ['浙',
  '江',
  '省',
  '宁',
  '波',
  '市',
  '鄞',
  '州',
  '区',
  '钟',
  '公',
  '庙',
  '街',
  '道',
  '保',
  '泉',
  '路',
  '1030',
  '号',
  '惠',
  '和',
  '大',
  '厦',
  'unk'],
 ['浙',
  '江',
  '省',
  '杭',
  '州',
  '市',
  '下',
  '城',
  '区',
  '朝',
  '晖',
  '路',
  '国',
  '都',
  '发',

In [26]:
final = []
for sen in biglist:
    scores, path = Nviterbi(7,lab,e,q,sen)
    for i in path:
        if i != '***START***':
            final.append(i)
    

In [27]:
f = open("/Users/khai/Home Documents/Term 6/Machine Learning/AL/dev.p4.out","w+", encoding="utf8")

for i in range(len(final)):
    if final[i] != '***STOP***':
        f.write(test[i]+' '+final[i]+'\n')
    else:
        f.write('\n')
        
f.close()